1. 31 Mayıs tarihi için 00:00 – 23:59 arası için

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from statsmodels.tsa.arima.model import ARIMA
import math
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import tree
import pandas as pd
from sklearn.ensemble import IsolationForest

In [17]:
train_df = pd.read_csv('/content/drive/MyDrive/innova/new_train.csv')


train_df['DATETIME'] = pd.to_datetime(train_df['DATETIME']).dt.tz_localize(None)  # Zaman dilimini kaldır

label_encoder = LabelEncoder()
train_df['SERVER'] = label_encoder.fit_transform(train_df['SERVER'])

#label_encoder_cpu = LabelEncoder()
#train_df['CPU'] = label_encoder_cpu.fit_transform(train_df['CPU'])


start_date = pd.to_datetime('2020-05-15')
end_date = pd.to_datetime('2020-05-30')
train_df_1 = train_df.loc[train_df['DATETIME'] < start_date]
test_df_1 = train_df.loc[(train_df['DATETIME'] >= start_date) & (train_df['DATETIME'] <= end_date)]

X_train = train_df_1[['DATETIME', 'SERVER']]
y_train = train_df_1['CPULOAD']
X_test = train_df_1[['DATETIME', 'SERVER']]
y_test = train_df_1['CPULOAD']

X_train['DATETIME'] = pd.to_datetime(X_train['DATETIME'])
X_train['DATETIME'] = X_train['DATETIME'].astype('int64') // 10**9  # UNIX zamanına dönüşümü

X_test['DATETIME'] = pd.to_datetime(X_test['DATETIME'])
X_test['DATETIME'] = X_test['DATETIME'].astype('int64')


<ipython-input-17-f0ecf03b75a1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['DATETIME'] = pd.to_datetime(X_train['DATETIME'])
<ipython-input-17-f0ecf03b75a1>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['DATETIME'] = X_train['DATETIME'].astype('int64') // 10**9  # UNIX zamanına dönüşümü
<ipython-input-17-f0ecf03b75a1>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# CSV dosyasını oku
#df = pd.read_csv('/content/drive/MyDrive/innova/new_train.csv')

# Veri ön işleme
#df['DATETIME'] = pd.to_datetime(df['DATETIME'])
#df['DATE'] = df['DATETIME'].dt.date
train_df_1 = train_df_1[['DATETIME', 'CPULOAD']].rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})
test_df_1 = test_df_1[['DATETIME', 'CPULOAD']].rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})


# Verileri eğitim ve test kümelerine ayır
#train, test = train_test_split(df, test_size=0.2, random_state=42)

# Veri ölçeklendirme
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df_1[['y']])
test_scaled = scaler.transform(test_df_1[['y']])

# Veriyi LSTM modeline uygun hale getirme
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 24  # Zaman adımları sayısı
X_train, y_train = create_dataset(train_scaled, train_scaled, time_steps)
X_test, y_test = create_dataset(test_scaled, test_scaled, time_steps)

# LSTM modelini oluştur
model = keras.Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğit
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Tahminleri yap
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

# RMSE değerini hesapla
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print('RMSE:', rmse)


Epoch 1/100
311/311 [==============================] - 15s 42ms/step - loss: 0.0125
Epoch 2/100
311/311 [==============================] - 13s 41ms/step - loss: 0.0022
Epoch 3/100
311/311 [==============================] - 13s 42ms/step - loss: 0.0012
Epoch 4/100
311/311 [==============================] - 13s 42ms/step - loss: 0.0011
Epoch 5/100
311/311 [==============================] - 13s 42ms/step - loss: 0.0011
Epoch 6/100
311/311 [==============================] - 13s 42ms/step - loss: 0.0010
Epoch 7/100
311/311 [==============================] - 13s 42ms/step - loss: 0.0010
Epoch 8/100
311/311 [==============================] - 13s 43ms/step - loss: 9.3297e-04
Epoch 9/100
311/311 [==============================] - 13s 43ms/step - loss: 9.6235e-04
Epoch 10/100
311/311 [==============================] - 14s 45ms/step - loss: 9.2049e-04
Epoch 11/100
311/311 [==============================] - 13s 42ms/step - loss: 9.2211e-04
Epoch 12/100
311/311 [==============================] - 15

In [8]:
y_pred

array([[ 2.8703082],
       [ 3.7973228],
       [ 2.9390316],
       ...,
       [11.169855 ],
       [ 5.979897 ],
       [10.6662035]], dtype=float32)

prophet

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


df = pd.read_csv('/content/drive/MyDrive/innova/train.csv')


df['DATETIME'] = pd.to_datetime(df['DATETIME']).dt.tz_localize(None)  # Zaman dilimini kaldır
df['DATE'] = df['DATETIME'].dt.date
df = df[['DATETIME', 'SERVER', 'CPU', 'CPULOAD']].rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})


train, test = train_test_split(df, test_size=0.2, random_state=42)

model = Prophet()


model.fit(train)

future = model.make_future_dataframe(periods=len(test), freq='15min')
forecast = model.predict(future)
forecast = forecast[['ds', 'yhat']]
forecast = forecast.rename(columns={'yhat': 'Predicted CPULOAD'})

y_true = test['y'].values
y_pred = forecast['Predicted CPULOAD'].tail(len(test)).values
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

output = pd.DataFrame({'RMSE': [rmse]})
print(output)


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmr3oqiss/n05dczu_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmr3oqiss/vqkoqbpg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98945', 'data', 'file=/tmp/tmpmr3oqiss/n05dczu_.json', 'init=/tmp/tmpmr3oqiss/vqkoqbpg.json', 'output', 'file=/tmp/tmpmr3oqiss/prophet_model3ds_jgiq/prophet_model-20230707145836.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:58:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:00:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


        RMSE
0  31.225125
